In [ ]:
!pip install -U accelerate==0.29.3 peft==0.10.0 bitsandbytes==0.43.1 transformers==4.44.2 trl==0.8.6 datasets==2.19.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [ ]:
import os
import torch
from datasets import load_dataset

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

if 'COLAB_GPU' in os.environ:
    !huggingface-cli login
else:
    notebook_login()

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!pip install jsonlines

In [ ]:
!git clone https://github.com/poloclub/Fine-tuning-LLMs.git

if 'COLAB_GPU' in os.environ:
    dir_path = '/content/Fine-tuning-LLMs/data/'
else:
    dir_path = 'Fine-tuning-LLMs/data/'

os.chdir(dir_path)

####
import jsonlines

def txt_to_jsonl(txt_file_path, jsonl_file_path):
  with open(txt_file_path, 'r') as txt_file, open(jsonl_file_path, 'w') as jsonl_file:
    for line in txt_file:
      jsonl_file.write(line)

txt_to_jsonl('hawaii_wf_1.txt', 'hawaii_wf_1.jsonl')
txt_to_jsonl('hawaii_wf_2.txt', 'hawaii_wf_2.jsonl')
txt_to_jsonl('hawaii_wf_3.txt', 'hawaii_wf_3.jsonl')
txt_to_jsonl('hawaii_wf_4.txt', 'hawaii_wf_4.jsonl')
txt_to_jsonl('hawaii_wf_5.txt', 'hawaii_wf_5.jsonl')
txt_to_jsonl('hawaii_wf_6.txt', 'hawaii_wf_6.jsonl')
txt_to_jsonl('hawaii_wf_7.txt', 'hawaii_wf_7.jsonl')
txt_to_jsonl('hawaii_wf_8.txt', 'hawaii_wf_8.jsonl')
txt_to_jsonl('hawaii_wf_9.txt', 'hawaii_wf_9.jsonl')
txt_to_jsonl('hawaii_wf_10.txt', 'hawaii_wf_10.jsonl')

train_dataset = load_dataset('jsonl', data_files={"train":[
    "hawaii_wf_1.jsonl", "hawaii_wf_2.jsonl",
    "hawaii_wf_3.jsonl","hawaii_wf_4.jsonl",
    "hawaii_wf_5.jsonl","hawaii_wf_6.jsonl",
    "hawaii_wf_7.jsonl","hawaii_wf_8.jsonl",
    "hawaii_wf_9.jsonl","hawaii_wf_10.jsonl"
]}, split='train')

####

train_dataset = load_dataset("text", data_files={"train":
                ["hawaii_wf_1.txt", "hawaii_wf_2.txt",
                 "hawaii_wf_3.txt","hawaii_wf_4.txt",
                 "hawaii_wf_5.txt","hawaii_wf_6.txt",
                 "hawaii_wf_7.txt","hawaii_wf_8.txt",
                "hawaii_wf_9.txt","hawaii_wf_10.txt","hawaii_wf_11.txt"]}, split='train')

####

os.chdir('..')
os.chdir('..')

Cloning into 'Fine-tuning-LLMs'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 47 (delta 14), reused 29 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (47/47), 9.34 MiB | 17.81 MiB/s, done.
Resolving deltas: 100% (14/14), done.


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# Hugging Face Basic Model 한국어 모델
# base_model = "teddylee777/Llama-3-Open-Ko-8B-gguf"  # 테디님의 Llama3 한국어 파인튜닝 모델
base_model = "meta-llama/Meta-Llama-3.1-8B-Instruct"               # beomi님의 Llama3 한국어 파인튜닝 모델

# 주가 증권 보고서 gemini 데이터셋
# hkcode_dataset = "uiyong/gemini_result_kospi_0517_jsonl"
hkcode_dataset = train_dataset

# 새로운 모델 이름
new_model = "Llama3.1-test1"

In [ ]:
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install -qqq flash-attn
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 64.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
# QLoRA config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=False,
)

In [ ]:
# llama 데이터 로드
# dataset = load_dataset(hkcode_dataset, split="train")
dataset = hkcode_dataset

# 데이터 확인
print( dataset[5] )

{'text': 'FAX: (808) 244-6411'}


In [ ]:
# !pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 116.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.1
    Uninstalling transformers-4.40.1:
      Successfully uninstalled transformers-4.40.1


In [ ]:
### 모델 로드
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [ ]:
# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(
              base_model,
              trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=10,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

In [ ]:
# 파인튜닝
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/1939 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

Step,Training Loss
25,4.187600
50,4.885100
75,3.378800
100,4.115100
125,3.271300
150,4.174400
175,3.108000
200,3.791200
225,3.051700
250,3.847300


TrainOutput(global_step=4850, training_loss=2.0706684887286313, metrics={'train_runtime': 2004.9585, 'train_samples_per_second': 9.671, 'train_steps_per_second': 2.419, 'total_flos': 1.6260667467005952e+16, 'train_loss': 2.0706684887286313, 'epoch': 10.0})

In [ ]:
logging.set_verbosity(logging.CRITICAL)

prompt = "삼성전자의 5월 17일 증권 현황은 어떤가요?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] 삼성전자의 5월 17일 증권 현황은 어떤가요? [/INST] [/s] [/a] [/t] [/d] [/p] [/m] [/x] [/u] [/f] [/v] [/r] [/i] [/c] [/e] [/a] [/t] [/u] [/f] [/v] [/r] [/i] [/c] [/e] [/a] [/t] [/u] [/f] [/v] [/r] [/i] [/c] [/e] [/a] [/t] [/u] [/f] [/v] [/r] [/i] [/c] [/e] [/a] [/t] [/u] [/f] [/v] [/r] [/i] [/c] [/e] [/a] [/t] [/u] [/f] [/v] [/r] [/i] [/c] [/


In [ ]:
logging.set_verbosity(logging.CRITICAL)

prompt = "Just answer this question: Tell me about the role of Maui Emergency Management Agency (MEMA) in the 2023 wildfires."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Just answer this question: Tell me about the role of Maui Emergency Management Agency (MEMA) in the 2023 wildfires. [/INST]A Maui resident shared that MEMA played a key role in coordinating government response to the 2023 Lahaina fires. They worked with other agencies to establish a joint information services (JIS) to share information and coordinate responses. They helped establish a joint emergency response system to coordinate responses to the fires. They also helped establish a joint system for sharing information on the fires, including sharing data on the number of cases and the types of cases. The resident emphasized that MEMA played a key role in coordinating the response to the fires, including establishing a joint system for sharing information on the fires and the types of cases. The resident noted that MEMA helped establish a joint system for sharing information on the fires on social media, including sharing data on the number of cases and the types of cases. The

In [ ]:
# ### ENTER YOUR QUESTION BELOW

# question = "Just answer this question: Tell me about the role of Maui Emergency Management Agency (MEMA) in the 2023 wildfires??"

# # Format the question
# eval_prompt = f"{question}\n\n"

# promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

# modelFinetuned.eval()
# with torch.no_grad():
#     print(tokenizer.decode(modelFinetuned.generate(**promptTokenized, max_new_tokens = 1024)[0], skip_special_tokens=True))
# torch.cuda.empty_cache()